<a href="https://colab.research.google.com/github/JvMotinha/Desafios/blob/main/Recompra_Bazica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [128]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import unicodedata as ucd

In [129]:
# Lendo o arquivo
base_vendas = pd.read_csv('/content/vendas_de_produtos.csv')

In [130]:
# Padronizando em letras minúsculas
base_vendas['Descrição_Produto'] = base_vendas['Descrição_Produto'].str.lower()

# Removendo acentuações
base_vendas['Descrição_Produto'] = base_vendas['Descrição_Produto'].apply(lambda x: ucd.normalize('NFKD', x).encode('ASCII', 'ignore').decode('ASCII'))


In [131]:
# 'atributos_base_vendas' armazena uma lista com os atributos da base de dados 'base_vendas'
atributos_base_vendas = base_vendas.columns.tolist()

# Exibe a lista com os atributos da 'base_vendas'
print(atributos_base_vendas)

['ID_Cliente', 'Data', 'ID_Produto', 'Descrição_Produto', 'Quantidade', 'Preço_Unitário', 'ID_Pedido', 'Desconto', 'Frete', 'Total_do_Pedido']


In [132]:
# Quantidade de atributos faltantes
for atributo in atributos_base_vendas:
  # 'qtd_atributo_faltante' armazena a quantidade de dados faltantes do atributo em questãp
  qtd_atributo_faltante = len(base_vendas.loc[pd.isnull(base_vendas[atributo])].index)
  print(f"{atributo}: {qtd_atributo_faltante}")

ID_Cliente: 2
Data: 0
ID_Produto: 403
Descrição_Produto: 0
Quantidade: 0
Preço_Unitário: 224
ID_Pedido: 0
Desconto: 44
Frete: 0
Total_do_Pedido: 0


In [133]:
# Identificar o 2 índices dos registros com atributo 'ID_Cliente' faltando
registros_ID_Cliente_faltante = base_vendas.loc[pd.isnull(base_vendas['ID_Cliente'])].index
registros_ID_Cliente_faltante

Int64Index([483, 485], dtype='int64')

In [134]:
# Removendo os dois registros com valores de ID_Cliente faltante 
base_vendas = base_vendas.drop(registros_ID_Cliente_faltante)

In [135]:
# Retorna a quantidade de registros com o atributo 'ID_Cliente' faltante
len(base_vendas.loc[pd.isnull(base_vendas['ID_Cliente'])].index)

0

In [136]:
# Variácel que armazena os clientes únicos
registros_ID_Cliente_unicos = np.unique(base_vendas['ID_Cliente'])
print(len(registros_ID_Cliente_unicos))

3783


In [137]:
# Cores das roupas coloridas
cores_roupas = ['areia', 'azul', 'berry', 'blue', 'bordo', 'cinza', 'coral', 'goiaba', 'grafite', 'lavanda', 'midnight', 'ocean', 'off white', 'pink', 'red', 'roze', 'salmon', 'toffe', 'uva', 'verde', 'vinho', 'wave']
print(cores_roupas)

['areia', 'azul', 'berry', 'blue', 'bordo', 'cinza', 'coral', 'goiaba', 'grafite', 'lavanda', 'midnight', 'ocean', 'off white', 'pink', 'red', 'roze', 'salmon', 'toffe', 'uva', 'verde', 'vinho', 'wave']


In [138]:
# Vetor com registros finals de todos os clientes
vetor_data_base = []

for cliente in registros_ID_Cliente_unicos:
  # Argumento para função 'query'
  arg = 'ID_Cliente == '
  arg_completo = arg + str(cliente)

  # 'base_cliente_especifico' é um Dataframe do cliente em questão
  base_cliente_especifico = base_vendas.query(arg_completo)

  # Criação da Coluna 'Peça_Colorida' que armazena 1 se o produto é colorido ou 0 caso contrário
  base_cliente_especifico['Peça_Colorida'] = base_cliente_especifico['Descrição_Produto'].apply(lambda x: 1 if any(word in x for word in cores_roupas) else 0)

  # Excluindo colunas ('ID_Produto', 'Descrição_Produto', 'Preço_Unitário', 'Desconto', 'Frete')
  base_cliente_especifico = base_cliente_especifico.drop(columns = ['ID_Produto',  'Descrição_Produto', 'Preço_Unitário', 'Desconto', 'Frete'])

  # Agrupamento 1: 'ID_Cliente', 'Data', 'ID_Pedido' | 'Quantidade': sum , 'Peça_Colorida': sum, 'Total_do_Pedido': 'mean'
  # Agrupo por pedido, somando a quantidade de produtos, somando a quantidade de peças coloridas, e mantendo o valor total do pedido 
  base_cliente_especifico = base_cliente_especifico.groupby(['ID_Cliente', 'Data', 'ID_Pedido'], as_index=False).agg({'Quantidade': 'sum', 'Peça_Colorida': 'sum', 'Total_do_Pedido': 'mean'})

  # Agrupamento 2: 'ID_Cliente', 'Data' | 'Quantidade': sum, 'Pela_Colorida': sum, 'Total_do_Pedido': sum
  # Agrupo por data, somando a quantidade de protutos, somando a quantidade de peças coloridas, somando os totais dos pedidos
  base_cliente_especifico = base_cliente_especifico.groupby(['ID_Cliente', 'Data'], as_index=False)['Quantidade', 'Peça_Colorida', 'Total_do_Pedido'].sum()


  # Vetor que armazena o registro do cliente específico
  registro_cliente = []

  # ID do Cliente
  id_cliente = base_cliente_especifico.loc[0,'ID_Cliente']
  registro_cliente.append(id_cliente)

  # Quantidade de Pedidos
  qtd_pedidos = len(np.unique(base_cliente_especifico['Data']))
  registro_cliente.append(qtd_pedidos)

  # Quantidade Total de Produtos
  qtd_total_produtos = base_cliente_especifico['Quantidade'].sum()
  registro_cliente.append(qtd_total_produtos)

  # Quantidade Total de Produtos Coloridos
  qtd_produtos_coloridos = base_cliente_especifico['Peça_Colorida'].sum()
  registro_cliente.append(qtd_produtos_coloridos)

  # LTV do cliente
  ltv_cliente = base_cliente_especifico['Total_do_Pedido'].sum()
  registro_cliente.append(ltv_cliente)

  # Ticket Médio do cliente
  ticket_medio_cliente = ltv_cliente/qtd_pedidos
  registro_cliente.append(ticket_medio_cliente)

  # Adiciona o registro ao vetor que será transformado no dataframe com todos os registros
  vetor_data_base.append(registro_cliente)


A saída de streaming foi truncada nas últimas 5000 linhas.
<ipython-input-138-6fdc69f97803>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_cliente_especifico['Peça_Colorida'] = base_cliente_especifico['Descrição_Produto'].apply(lambda x: 1 if any(word in x for word in cores_roupas) else 0)
<ipython-input-138-6fdc69f97803>:24: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  base_cliente_especifico = base_cliente_especifico.groupby(['ID_Cliente', 'Data'], as_index=False)['Quantidade', 'Peça_Colorida', 'Total_do_Pedido'].sum()
<ipython-input-138-6fdc69f97803>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

In [139]:
# Exemplo
vetor_data_base[109]

[12268398905.0, 43, 90, 54, 11313.900000000001, 263.11395348837215]

In [140]:
# Criando o DataFrame com todos os registros dos clientes
df_registros_clientes = pd.DataFrame(vetor_data_base)
df_registros_clientes

,0,1,2,3,4,5
0,1.000000e+00,1,1,0,113.90,113.90
1,2.000000e+00,1,3,0,301.90,301.90
2,3.000000e+00,1,1,0,133.97,133.97
3,4.000000e+00,1,2,0,192.90,192.90
4,5.000000e+00,1,1,0,123.90,123.90
...,...,...,...,...,...,...
3778,1.599063e+10,1,2,0,209.44,209.44
3779,1.599064e+10,1,2,0,209.44,209.44
3780,1.599065e+10,1,2,2,214.20,214.20
3781,1.599067e+10,1,1,0,140.19,140.19


In [141]:
# Renomeando as colunas do DataFrame para melhor interpretação
df_registros_clientes = df_registros_clientes.rename(columns={0: 'ID_Cliente', 1: 'Qtd_Pedidos', 2: 'Qtd_Total_Produtos', 3: 'Qtd_Produtos_Coloridos', 4: 'LTV', 5: 'Ticket_Medio'})
df_registros_clientes

,ID_Cliente,Qtd_Pedidos,Qtd_Total_Produtos,Qtd_Produtos_Coloridos,LTV,Ticket_Medio
0,1.000000e+00,1,1,0,113.90,113.90
1,2.000000e+00,1,3,0,301.90,301.90
2,3.000000e+00,1,1,0,133.97,133.97
3,4.000000e+00,1,2,0,192.90,192.90
4,5.000000e+00,1,1,0,123.90,123.90
...,...,...,...,...,...,...
3778,1.599063e+10,1,2,0,209.44,209.44
3779,1.599064e+10,1,2,0,209.44,209.44
3780,1.599065e+10,1,2,2,214.20,214.20
3781,1.599067e+10,1,1,0,140.19,140.19


In [142]:
# Colunas para serem normalizadas
colunas_para_normalizacao = ['Qtd_Pedidos', 'Qtd_Total_Produtos', 'Qtd_Produtos_Coloridos', 'LTV', 'Ticket_Medio']

In [143]:
# Importa Biblioteca
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [144]:
# Lista com os registros normalizados
atributos_normalizados = scaler.fit_transform(df_registros_clientes[colunas_para_normalizacao])
# print(atributos_normalizados)

In [145]:
# Transforma a lista com os registros normalizados em um DataFrame Pandas
df_atributos_normalizados = pd.DataFrame(atributos_normalizados)
df_atributos_normalizados

,0,1,2,3,4
0,-0.481263,-0.481307,-0.685111,-0.522346,-0.281875
1,-0.481263,-0.228086,-0.685111,-0.279199,0.150190
2,-0.481263,-0.481307,-0.685111,-0.496389,-0.235750
3,-0.481263,-0.354697,-0.685111,-0.420173,-0.100316
4,-0.481263,-0.481307,-0.685111,-0.509413,-0.258893
...,...,...,...,...,...
3778,-0.481263,-0.354697,-0.685111,-0.398781,-0.062303
3779,-0.481263,-0.354697,-0.685111,-0.398781,-0.062303
3780,-0.481263,-0.354697,-0.077640,-0.392625,-0.051363
3781,-0.481263,-0.481307,-0.685111,-0.488344,-0.221455


In [146]:
# Cria um novo DataFrame que é a copia do df_registros_clientes, mas agora com os valores dos atributos normalizados
df_registros_clientes_normalizados = df_registros_clientes
df_registros_clientes_normalizados['Qtd_Pedidos'] = df_atributos_normalizados[0]
df_registros_clientes_normalizados['Qtd_Total_Produtos'] = df_atributos_normalizados[1]
df_registros_clientes_normalizados['Qtd_Produtos_Coloridos'] = df_atributos_normalizados[2]
df_registros_clientes_normalizados['LTV'] = df_atributos_normalizados[3]
df_registros_clientes_normalizados['Ticket_Medio'] = df_atributos_normalizados[4]
df_registros_clientes_normalizados

,ID_Cliente,Qtd_Pedidos,Qtd_Total_Produtos,Qtd_Produtos_Coloridos,LTV,Ticket_Medio
0,1.000000e+00,-0.481263,-0.481307,-0.685111,-0.522346,-0.281875
1,2.000000e+00,-0.481263,-0.228086,-0.685111,-0.279199,0.150190
2,3.000000e+00,-0.481263,-0.481307,-0.685111,-0.496389,-0.235750
3,4.000000e+00,-0.481263,-0.354697,-0.685111,-0.420173,-0.100316
4,5.000000e+00,-0.481263,-0.481307,-0.685111,-0.509413,-0.258893
...,...,...,...,...,...,...
3778,1.599063e+10,-0.481263,-0.354697,-0.685111,-0.398781,-0.062303
3779,1.599064e+10,-0.481263,-0.354697,-0.685111,-0.398781,-0.062303
3780,1.599065e+10,-0.481263,-0.354697,-0.077640,-0.392625,-0.051363
3781,1.599067e+10,-0.481263,-0.481307,-0.685111,-0.488344,-0.221455


In [147]:
# Pesos dos atributos
# Acabei não usando o 'Qtd_Produtos_Coloridos'
peso_pedidos = 10
peso_produtos = 2
peso_ltv = 3
peso_ticket_medio = 3

In [148]:
# Criação do 'df_registros_clientes_alvo' que possui a coluna 'Valor_Objetivo', que representa uma valor proporcional a probabilidade de compra
df_registros_clientes_alvo = df_registros_clientes_normalizados
# o atributo 'Valor_Objetivo' segue a seguinte função:
df_registros_clientes_alvo['Valor_Objetivo'] = (peso_pedidos * df_registros_clientes_normalizados['Qtd_Pedidos'] + peso_produtos * df_registros_clientes_normalizados['Qtd_Total_Produtos'] + peso_ltv * df_registros_clientes_normalizados['LTV'] + peso_ticket_medio * df_registros_clientes_normalizados['Ticket_Medio'])
df_registros_clientes_normalizados

,ID_Cliente,Qtd_Pedidos,Qtd_Total_Produtos,Qtd_Produtos_Coloridos,LTV,Ticket_Medio,Valor_Objetivo
0,1.000000e+00,-0.481263,-0.481307,-0.685111,-0.522346,-0.281875,-8.187910
1,2.000000e+00,-0.481263,-0.228086,-0.685111,-0.279199,0.150190,-5.655832
2,3.000000e+00,-0.481263,-0.481307,-0.685111,-0.496389,-0.235750,-7.971662
3,4.000000e+00,-0.481263,-0.354697,-0.685111,-0.420173,-0.100316,-7.083491
4,5.000000e+00,-0.481263,-0.481307,-0.685111,-0.509413,-0.258893,-8.080163
...,...,...,...,...,...,...,...
3778,1.599063e+10,-0.481263,-0.354697,-0.685111,-0.398781,-0.062303,-6.905278
3779,1.599064e+10,-0.481263,-0.354697,-0.685111,-0.398781,-0.062303,-6.905278
3780,1.599065e+10,-0.481263,-0.354697,-0.077640,-0.392625,-0.051363,-6.853991
3781,1.599067e+10,-0.481263,-0.481307,-0.685111,-0.488344,-0.221455,-7.904644


In [149]:
# Criação de um DataFrame com os valores do 'Valor_Objetivo' ordenados de forma decrescente
df_registros_clientes_alvo_sort = df_registros_clientes_alvo.sort_values('Valor_Objetivo', ascending=False)
df_registros_clientes_alvo_sort

,ID_Cliente,Qtd_Pedidos,Qtd_Total_Produtos,Qtd_Produtos_Coloridos,LTV,Ticket_Medio,Valor_Objetivo
2655,1.591419e+10,-0.481263,40.413874,0.833566,30.087903,54.111667,328.613826
109,1.226840e+10,16.413567,10.787024,15.716601,13.962999,0.061051,227.781869
331,1.408880e+10,11.586473,6.735489,7.819481,8.768063,-0.002656,155.631925
121,1.242904e+10,9.977441,5.595995,4.174656,6.200277,-0.091506,129.292715
184,1.273229e+10,9.575183,4.709722,6.908274,5.004097,-0.155869,119.715963
...,...,...,...,...,...,...,...
911,1.560540e+10,-0.481263,-0.481307,-0.685111,-0.669657,-0.543642,-9.415143
914,1.560541e+10,-0.481263,-0.481307,-0.685111,-0.669657,-0.543642,-9.415143
1098,1.565507e+10,-0.481263,-0.481307,-0.685111,-0.669657,-0.543642,-9.415143
2158,1.583565e+10,-0.481263,-0.481307,-0.381375,-0.669657,-0.543642,-9.415143


In [150]:
# Os 100 registros com maiores 'Valor_Objetivo'
registros_100 = df_registros_clientes_alvo_sort.head(100)
registros_100

,ID_Cliente,Qtd_Pedidos,Qtd_Total_Produtos,Qtd_Produtos_Coloridos,LTV,Ticket_Medio,Valor_Objetivo
2655,1.591419e+10,-0.481263,40.413874,0.833566,30.087903,54.111667,328.613826
109,1.226840e+10,16.413567,10.787024,15.716601,13.962999,0.061051,227.781869
331,1.408880e+10,11.586473,6.735489,7.819481,8.768063,-0.002656,155.631925
121,1.242904e+10,9.977441,5.595995,4.174656,6.200277,-0.091506,129.292715
184,1.273229e+10,9.575183,4.709722,6.908274,5.004097,-0.155869,119.715963
...,...,...,...,...,...,...,...
1712,1.576748e+10,2.736800,1.417850,1.744772,2.360626,0.054661,37.449555
1256,1.569073e+10,2.736800,1.797681,3.870920,2.142880,0.011669,37.427005
1509,1.574602e+10,2.736800,1.924292,2.352243,2.015331,-0.013514,37.222030
1034,1.562245e+10,1.932284,3.317007,4.478391,3.073563,0.406586,36.397300


In [151]:
# A lista com os ID_Cliente dos 100 clientes com maiores 'Valor_Objetivo'
clientes_100 = registros_100['ID_Cliente'].tolist()
clientes_100

[15914194842.0,
 12268398905.0,
 14088799935.0,
 12429037473.0,
 12732289533.0,
 12429348490.0,
 15692298038.0,
 15662353010.0,
 15625006098.0,
 15606329684.0,
 14463206503.0,
 15788478852.0,
 12868213443.0,
 15533494121.0,
 15762013256.0,
 15357970634.0,
 12429148020.0,
 15606412869.0,
 13611623453.0,
 12429171179.0,
 13589205799.0,
 14853245762.0,
 15745637988.0,
 15787455011.0,
 15737982431.0,
 15758948769.0,
 15690425901.0,
 12868160682.0,
 15816891950.0,
 14849351530.0,
 12429382466.0,
 15588899895.0,
 15300613422.0,
 15805840884.0,
 103.0,
 15602583615.0,
 15694032200.0,
 12788493590.0,
 15689555963.0,
 15145762782.0,
 13816111407.0,
 15706960458.0,
 12429386125.0,
 15695312404.0,
 15771806158.0,
 15755711667.0,
 12574633308.0,
 12383774078.0,
 13022322113.0,
 15605402482.0,
 12971103152.0,
 13817352141.0,
 15858834205.0,
 13265389909.0,
 15031535201.0,
 15757467131.0,
 15602735269.0,
 12385092320.0,
 13031151143.0,
 14358427801.0,
 14508214784.0,
 15388066216.0,
 14852448230.0,
